# 层与块
+ 块（block）可以描述单个层、由多个层组成的组件或整个模型本⾝。使⽤块进⾏抽象的⼀个好处是可以将⼀些块组合成更⼤的组件。+ 从编程的⻆度来看，块由类（class）表⽰。它的任何⼦类都必须定义⼀个将其输⼊转换为输出的前向传播函数，并且必须存储任何必需的参数。注意，有些块不需要任何参数。最后，为了计算梯度，块必须具有反向传播函数。

## 1.自定义块
+ 每个块必须有的基本功能
    + 1. 将输⼊数据作为其前向传播函数的参数。
    + 2. 通过前向传播函数来⽣成输出。请注意，输出的形状可能与输⼊的形状不同。例如，我们上⾯模型中的第⼀个全连接的层接收⼀个20维的输⼊，但是返回⼀个维度为256的输出。
    + 3. 计算其输出关于输⼊的梯度，可通过其反向传播函数进⾏访问。通常这是⾃动发⽣的。
    + 4. 存储和访问前向传播计算所需的参数。
    + 5. 根据需要初始化模型参数。


In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
# MLP类继承了表⽰块的类。我们的实现只需要提供我们⾃⼰的构造函数（Python中的__init__函数）和前向传播函数。
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
    
    # 定义前向传播
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0273, -0.0458,  0.3154, -0.1051, -0.1642, -0.0974,  0.0413,  0.1286,
         -0.0888,  0.1153],
        [ 0.0981,  0.0688,  0.2746, -0.1930, -0.0117, -0.1888,  0.2200,  0.2125,
         -0.0235,  0.1447]], grad_fn=<AddmmBackward0>)

## 2.顺序块
+ 需要定义两个关键函数
    + 1. ⼀种将块逐个追加到列表中的函数；
    + 2. ⼀种前向传播函数，⽤于将输⼊按追加块的顺序传递给块组成的“链条”。


In [8]:
# 个人理解顺序块：就是将每一步计算的步骤类型模块写在了初始化网络时的参数部分，每次如果网络结构有变化，就是在初始网络类的时候变化
# 而自定义块的计算步骤就是在类内部就根据需求就写好了，新建模块时内部为黑盒
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里的module就是一个具体的块的实例，定义好后按顺序添加合成一个大的块（网络）
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    # 前向计算模块
    def forward(self, X):
        for block in self._modules.values():
            # 按照加入的顺序将数据放入这些块中进行计算
            X = block(X)
        return X
            
        

In [9]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0783,  0.0649, -0.0165, -0.1007,  0.0299, -0.2609, -0.0553, -0.2416,
          0.0603, -0.1872],
        [ 0.1472,  0.0259, -0.0645, -0.0537,  0.0465, -0.1875, -0.1480, -0.1474,
          0.0537, -0.1894]], grad_fn=<AddmmBackward0>)

## 3. 在前向传播函数中执行代码
+ Sequential类使模型构造变得简单，允许我们组合新的架构，⽽不必定义⾃⼰的类。然⽽，并不是所有的架构都是简单的顺序架构。当需要更强的灵活性时，我们需要定义⾃⼰的块。

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    # 在前向传播中添加功能
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 矩阵函数
        # 复用全连接层，共享参数
        X = self.linear(X)
        # 控制流（额外功能）
        while X.abs().sum() > 1: 
            X /= 2
        return X.sum()

## 4.混合块：自定义块与顺序块结合

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        # 确定该块的一些输入格式，计算结构
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), # hidden1
            nn.ReLU(), # activation1
            nn.Linear(64, 32), # hidden2
            nn.ReLU(), # activation2
        )
        self.linear = nn.Linear(32, 16) # fc
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(),
                       nn.Linear(16, 20),
                       FixedHiddenMLP())
chimera(X)
    

tensor(-0.3164, grad_fn=<SumBackward0>)

## 5. 总结
+ ⼀个块可以由许多层组成；⼀个块可以由许多块组成。
+ 块可以包含代码。
+ 块负责⼤量的内部处理，包括参数初始化和反向传播。
+ 层和块的顺序连接由Sequential块处理。


# 参数管理
+ 访问参数，⽤于调试、诊断和可视化
+ 参数初始化
+ 在不同模型间共享参数

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.4045],
        [0.4666]], grad_fn=<AddmmBackward0>)

## 1. 参数访问

In [13]:
# 访问最后一个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2276, -0.1335, -0.2281, -0.1108, -0.0979,  0.3284,  0.3434,  0.0377]])), ('bias', tensor([0.2230]))])


In [14]:
# 访问目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2230], requires_grad=True)
tensor([0.2230])


In [15]:
# 访问网络所有参数
# 第一个全连接层
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 所有的层
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4890],
        [-0.4890]], grad_fn=<AddmmBackward0>)

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 2.参数初始化

In [20]:
# 内置初始化
def init_normal(m):
    # 这里m表示某一层的参数
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0119, -0.0009, -0.0129,  0.0005]), tensor(0.))

In [21]:
# 自定义初始化
# nn.init.uniform_(m.weight, *, *)

## 3. 参数绑定
+ 可以定一个共享层，使用其参数来设置另外一个网络中层的参数

In [23]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


# 延后初始化
> 框架的延后初始化（defers initialization），即直到数第⼀次通过模型传递时，框架才会动态地推断出每个层的⼤⼩
+ (但是不知道为什么PDF上这一节无代码)

# 自定义层
> 我们可以⽤创造性的⽅式组合不同的层，从⽽设计出适⽤于各种任务的架构。

## 1. 不带参数的层


In [26]:
import torch 
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.9849e-09, grad_fn=<MeanBackward0>)

## 2. 带参数的层

In [30]:
class MyLinear(nn.Module):
    def __init__(self, inunits, units):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(inunits, units))
        self.bias = nn.Parameter(torch.rand(units, ))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [32]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[0.3969, 0.4178, 0.4165],
        [0.2438, 0.4554, 0.4774],
        [0.5350, 0.9869, 0.7798],
        [0.3520, 0.7862, 0.8062],
        [0.0415, 0.5501, 0.4565]], requires_grad=True)

In [33]:
linear(torch.rand(2, 5))

tensor([[0.9813, 2.0641, 2.4508],
        [1.0176, 2.1760, 2.5328]])